In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Release GPU memory
!pip install numba 
from numba import cuda 
device = cuda.get_current_device()
device.reset()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!rm -r "/content/drive/MyDrive/generated"
!mkdir "/content/drive/MyDrive/generated"

In [4]:
import numpy as np
import cv2
import os
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data
from google.colab.patches import cv2_imshow
from torchvision.models.inception import inception_v3
from scipy.stats import entropy
from glob import glob
from tqdm import tqdm
# https://github.com/eyalbetzalel/inception-score-pytorch/blob/master/inception_score.py
def inception_score(imgs, batch_size=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    batch_size -- batch size for feeding into Inception v3
    """

    N = len(imgs)

    if torch.cuda.is_available():
        dtype = torch.cuda.FloatTensor
    else:
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    py = np.mean(preds, axis=0)
    scores = []
    for i in range(preds.shape[0]):
        pyx = preds[i, :]
        scores.append(entropy(pyx, py))
    return np.exp(np.mean(scores))

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt

output_path = "/content/drive/MyDrive/generated"
model_path = "/content/drive/MyDrive/Disso/trained_models/DCGAN/cgan_generator.h5"
num_of_images_per_class = 100


model = tf.keras.models.load_model(model_path)
pred = model.predict([np.random.random(size=(num_of_images_per_class,100)),np.ones(shape = (num_of_images_per_class,1))])*255

for ind, img in enumerate(pred):
  cv2.imwrite("/{}/class1{}.png".format(output_path, ind), cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))

pred = model.predict([np.random.random(size=(num_of_images_per_class,100)),np.zeros(shape = (num_of_images_per_class,1))])*255
for ind, img in enumerate(pred):
  cv2.imwrite("/{}/class0{}.png".format(output_path, ind), cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)) 

In [6]:
!pip install fid_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.2 MB 34.7 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 4.1.4 requires scipy>=1.4.1, but you have scipy 1.3.2 which is incompatible.
plotnine 0.8.0 requires scipy>=1.5.0, but you have scipy 1.3.2 which is incompatible.
jaxlib 0.3.14+cuda11.cudnn805 requires scipy>=1.5, but you have scipy 1.3.2 which is incompatible.
jax 0.3.14 requires scipy>=1.5, but you have scipy 1.3.2 which is incompatible.
aeppl 0.0.33 requires scipy>=1.4.0, but you have scipy 1.3.2 which is incompatible.


In [7]:
from fid_score.fid_score import FidScore

original_data_path = "/content/drive/MyDrive/Disso/original_data"
generated_data_path = "/content/drive/MyDrive/generated"
scores = [FidScore([original_data_path,generated_data_path], batch_size=64).calculate_fid_score() for i in range(1)]
print("Scores:")
print(*[f"{score:.1f}" for score in scores],sep="\t")


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth


  0%|          | 0.00/91.2M [00:00<?, ?B/s]

  0%|          | 0/26 [00:00<?, ?it/s]

Present batch 1/64

  4%|▍         | 1/26 [00:37<15:33, 37.33s/it]

...

 75%|███████▌  | 3/4 [00:18<00:06,  6.22s/it]

done
Present batch 193/64

100%|██████████| 4/4 [00:19<00:00,  4.88s/it]

done


Scores:
314.3


In [8]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

from glob import glob
import cv2

from tqdm import tqdm

# https://github.com/eyalbetzalel/inception-score-pytorch/blob/master/inception_score.py
def inception_score(imgs, batch_size=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    batch_size -- batch size for feeding into Inception v3
    """
    N = len(imgs)

    if torch.cuda.is_available():
        dtype = torch.cuda.FloatTensor
    else:
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    py = np.mean(preds, axis=0)
    scores = []
    for i in range(preds.shape[0]):
        pyx = preds[i, :]
        scores.append(entropy(pyx, py))
    return np.exp(np.mean(scores))

imgs = torch.tensor([cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB) for file in glob(generated_data_path+"/*.png", recursive=True)])
imgs = torch.permute(imgs, (0,3,1,2)) / 128 - 1
inception_score(imgs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

100%|██████████| 200/200 [00:04<00:00, 49.18it/s]


1.00499147486076